In [9]:
!pip install scikit-learn


In [11]:
!pip install pymupdf
!pip install scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 48.1 MB/s eta 0:00:00


In [12]:
import os  # Module for interacting with the operating system
import fitz  # PyMuPDF
from sklearn.feature_extraction.text import TfidfVectorizer  # Module for text vectorization using TF-IDF
from sklearn.metrics.pairwise import cosine_similarity  # Module for calculating cosine similarity
from google.colab import files  # For file upload

# Upload PDF files
uploaded = files.upload()

student_files = list(uploaded.keys())

# Function to extract text from PDF
def extract_text_from_pdf(file_path):
    doc = fitz.open(file_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Extract text from each PDF file
student_notes = [extract_text_from_pdf(file) for file in student_files]

# Function to vectorize the text using TF-IDF
def vectorize(texts):
    vectorizer = TfidfVectorizer(stop_words='english')  # Handle stop words
    texts = [text for text in texts if text.strip()]  # Remove empty strings
    if not texts:
        raise ValueError("No valid documents to vectorize.")
    return vectorizer.fit_transform(texts).toarray()

# Function to calculate similarity between two documents
def similarity(doc1, doc2):
    return cosine_similarity([doc1, doc2])

# Vectorize the student notes using TF-IDF
vectors = vectorize(student_notes)
s_vectors = list(zip(student_files, vectors))
plagiarism_results = set()

# Function to check plagiarism among the student notes
def check_plagiarism():
    global s_vectors
    for student_a, text_vector_a in s_vectors:
        new_vectors = s_vectors.copy()
        current_index = new_vectors.index((student_a, text_vector_a))
        del new_vectors[current_index]
        for student_b, text_vector_b in new_vectors:
            # Calculate cosine similarity between two text vectors
            sim_score = similarity(text_vector_a, text_vector_b)[0][1]
            # Sort the student file names alphabetically to avoid duplicates
            student_pair = sorted((student_a, student_b))
            # Create a tuple with student file names and similarity score
            score = (student_pair[0], student_pair[1], sim_score)
            # Add the tuple to plagiarism_results set
            plagiarism_results.add(score)
    return plagiarism_results

# Print the plagiarism results
for data in check_plagiarism():
    print("Similarity data:\n", data)


Saving unit 3 gmm.pdf to unit 3 gmm (1).pdf
